<a href="https://colab.research.google.com/github/beejhay31/100-days-of-code/blob/master/Scaled_Hamoye_Quiz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
df=pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/00374/energydata_complete.csv')

In [ ]:
df.head(10)

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,RH_5,T6,RH_6,T7,RH_7,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.890000,47.596667,19.20,44.790000,19.79,44.730000,19.000000,45.566667,17.166667,55.200000,7.026667,84.256667,17.200000,41.626667,18.2,48.900000,17.033333,45.53,6.600000,733.500000,92.000000,7.000000,63.000000,5.300000,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.890000,46.693333,19.20,44.722500,19.79,44.790000,19.000000,45.992500,17.166667,55.200000,6.833333,84.063333,17.200000,41.560000,18.2,48.863333,17.066667,45.56,6.483333,733.600000,92.000000,6.666667,59.166667,5.200000,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.890000,46.300000,19.20,44.626667,19.79,44.933333,18.926667,45.890000,17.166667,55.090000,6.560000,83.156667,17.200000,41.433333,18.2,48.730000,17.000000,45.50,6.366667,733.700000,92.000000,6.333333,55.333333,5.100000,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.890000,46.066667,19.20,44.590000,19.79,45.000000,18.890000,45.723333,17.166667,55.090000,6.433333,83.423333,17.133333,41.290000,18.1,48.590000,17.000000,45.40,6.250000,733.800000,92.000000,6.000000,51.500000,5.000000,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.890000,46.333333,19.20,44.530000,19.79,45.000000,18.890000,45.530000,17.200000,55.090000,6.366667,84.893333,17.200000,41.230000,18.1,48.590000,17.000000,45.40,6.133333,733.900000,92.000000,5.666667,47.666667,4.900000,10.084097,10.084097
5,2016-01-11 17:50:00,50,40,19.890000,46.026667,19.20,44.500000,19.79,44.933333,18.890000,45.730000,17.133333,55.030000,6.300000,85.766667,17.133333,41.260000,18.1,48.590000,17.000000,45.29,6.016667,734.000000,92.000000,5.333333,43.833333,4.800000,44.919484,44.919484
6,2016-01-11 18:00:00,60,50,19.890000,45.766667,19.20,44.500000,19.79,44.900000,18.890000,45.790000,17.100000,54.966667,6.263333,86.090000,17.133333,41.200000,18.1,48.590000,17.000000,45.29,5.900000,734.100000,92.000000,5.000000,40.000000,4.700000,47.233763,47.233763
7,2016-01-11 18:10:00,60,50,19.856667,45.560000,19.20,44.500000,19.73,44.900000,18.890000,45.863333,17.100000,54.900000,6.190000,86.423333,17.100000,41.200000,18.1,48.590000,17.000000,45.29,5.916667,734.166667,91.833333,5.166667,40.000000,4.683333,33.039890,33.039890
8,2016-01-11 18:20:00,60,40,19.790000,45.597500,19.20,44.433333,19.73,44.790000,18.890000,45.790000,17.166667,55.000000,6.123333,87.226667,17.166667,41.400000,18.1,48.590000,17.000000,45.29,5.933333,734.233333,91.666667,5.333333,40.000000,4.666667,31.455702,31.455702
9,2016-01-11 18:30:00,70,40,19.856667,46.090000,19.23,44.400000,19.79,44.863333,18.890000,46.096667,17.100000,55.000000,6.190000,87.626667,17.200000,41.500000,18.1,48.590000,17.000000,45.29,5.950000,734.300000,91.500000,5.500000,40.000000,4.650000,3.089314,3.089314


In [ ]:
column_names = {'T1':'Temp_Kitch(Cel)', 'RH_1': 'Humid_kitch(%)',
'T2': 'Temp_liv_r(Cel)', 'RH_2': 'Humid_liv_r(%)', 'T3': 'Temp_laund_r(Cel)', 'RH_3': 'Humid_laund_r(%)',
'T4': 'Temp_office_r(Cel)', 'RH_4': 'Humid_office_r(%)', 'T5': 'Temp_bath_r(Cel)', 'RH_5': 'Humid_bath_r(%)', 
'T6': 'Temp_out_build_North(Cel)', 'RH_6': 'Humid_out_build_North(%)',
'T7': 'Temp_iron_r(Cel)', 'RH_7': 'Humid_iron_r(%)', 'T8': 'Temp_teen_r(Cel)', 'RH_8': 'Humid_teen_r(%)',
'T9': 'Humid_par_room(Cel)', 'RH_9': 'Humid_par_room(%)',
'rv1': 'rand_var1', 'rv2': 'rand_var2'}

In [ ]:
df = df.rename(columns=column_names)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
features_df = df.drop(columns=['date', 'lights'])
normalised_df = pd.DataFrame(scaler.fit_transform(features_df), columns=features_df.columns)
#features_df = normalised_df.drop(columns=['date', 'lights'])
target_var = normalised_df['Appliances']

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(normalised_df, target_var, test_size=0.3, random_state=42)

In [ ]:
from sklearn.linear_model import LinearRegression
linear_reg = LinearRegression()
linear_reg.fit(x_train, y_train)
predicted_values = linear_reg.predict(x_test)

In [ ]:
pred_val_df=pd.DataFrame(predicted_values)
pred_val_df

,0
0,0.028037
1,0.074766
2,0.037383
3,0.037383
4,0.056075
...,...
5916,0.224299
5917,0.046729
5918,0.037383
5919,0.093458


In [ ]:
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, predicted_values)
round(mae, 3)

0.0

In [ ]:
from sklearn.metrics import r2_score
r2_score = r2_score(y_test, predicted_values)
round(r2_score, 3)

1.0

In [ ]:
import numpy as np
rss = np.sum(np.square(y_test - predicted_values))
round(rss, 3)

0.0

In [ ]:
from sklearn.metrics import  mean_squared_error
rmse = np.sqrt(mean_squared_error(y_test, predicted_values))
round(rmse, 3)

0.0

In [ ]:
from sklearn.linear_model import Ridge
ridge_reg = Ridge(alpha=0.5)
ridge_reg.fit(x_train, y_train)

Ridge(alpha=0.5, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

In [ ]:
from sklearn.linear_model import Lasso
lasso_reg = Lasso(alpha=0.001)
lasso_reg.fit(x_train, y_train)

Lasso(alpha=0.001, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [ ]:
from sklearn.linear_model import ElasticNet
e_net = ElasticNet(alpha = 0.001) 
e_net.fit(x_train, y_train) 

ElasticNet(alpha=0.001, copy_X=True, fit_intercept=True, l1_ratio=0.5,
           max_iter=1000, normalize=False, positive=False, precompute=False,
           random_state=None, selection='cyclic', tol=0.0001, warm_start=False)

In [ ]:
def get_weights_df(model, feat, col_name):
  #this function returns the weight of every feature
  weights = pd.Series(model.coef_, feat.columns).sort_values()
  weights_df = pd.DataFrame(weights).reset_index()
  weights_df.columns = ['Features', col_name]
  weights_df[col_name].round(3)
  return weights_df

In [ ]:
linear_model_weights = get_weights_df(linear_reg, x_train, 'Linear_Model_Weight')
ridge_weights_df = get_weights_df(ridge_reg, x_train, 'Ridge_Weight')
lasso_weights_df = get_weights_df(lasso_reg, x_train, 'Lasso_weight')
e_weights_df = get_weights_df(e_net, x_train, 'E_weights')

In [ ]:
final_weights = pd.merge(linear_model_weights, ridge_weights_df, on='Features')
final_weights = pd.merge(final_weights, lasso_weights_df, on='Features')
final_weights=pd.merge(final_weights, e_weights_df, on='Features')

In [ ]:
final_weights

,Features,Linear_Model_Weight,Ridge_Weight,Lasso_weight,E_weights
0,Humid_liv_r(%),-3.188348e-16,-0.001818,-0.000000,-0.000000
1,Windspeed,-2.278562e-16,0.000138,0.000000,0.000000
2,Temp_liv_r(Cel),-2.054633e-16,-0.000879,0.000000,0.000000
3,Humid_office_r(%),-1.092237e-16,0.000110,0.000000,0.000000
4,Humid_laund_r(%),-9.242872e-17,0.000431,0.000000,0.000000
5,Humid_out_build_North(%),-8.979859e-17,0.000159,-0.000000,-0.000000
6,rand_var2,-8.315675e-17,0.000003,-0.000000,-0.000000
7,rand_var1,-8.315675e-17,0.000003,-0.000000,-0.000000
8,Temp_Kitch(Cel),-7.787265e-17,-0.000098,0.000000,0.000000
9,Visibility,-4.138629e-17,0.000055,0.000000,0.000000
